In [6]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
!pip install -U spacy

In [8]:
!python -m spacy init fill-config /content/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
from datasets import load_dataset

dataset = load_dataset("conll2003")  # Load CoNLL-2003
train_data = dataset["train"]
test_valid_data = dataset["test"].train_test_split(test_size=0.5)
test_data = test_valid_data["test"]
valid_data = test_valid_data["train"]
print(train_data[0])  # Check first sample

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] غ
The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [10]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
def prepare_date(data,name):
  nlp = spacy.blank("en")  # Start with a blank English model
  db = DocBin()

  for example in data:
      doc = nlp.make_doc(" ".join(example["tokens"]))  # Convert tokens to text
      ents = []
      for i, tag_id in enumerate(example["ner_tags"]):
          if tag_id != 0:  # Skip 'O' tags
              label = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"][tag_id]
              span = doc.char_span(
                  doc.text.find(example["tokens"][i]),  # Start position
                  doc.text.find(example["tokens"][i]) + len(example["tokens"][i]),  # End position
                  label=label
              )
              if span is not None:  # Handle tokenization mismatches
                  ents.append(span)
      doc.ents = filter_spans(ents)
      db.add(doc)

  db.to_disk(name)  # Save for training

In [11]:
prepare_date(valid_data,"train.spacy")
prepare_date(valid_data,"valid.spacy")
prepare_date(test_data,"test.spacy")

In [14]:
!python -m spacy train config.cfg \
    --output ./output \
    --paths.train ./train.spacy \
    --paths.dev ./valid.spacy \
    --verbose \
    --gpu-id 0

[2025-05-04 12:37:10,041] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-05-04 12:37:10,624] [INFO] Set up nlp object from config
[2025-05-04 12:37:10,639] [DEBUG] Loading corpus from path: valid.spacy
[2025-05-04 12:37:10,640] [DEBUG] Loading corpus from path: train.spacy
[2025-05-04 12:37:10,640] [INFO] Pipeline: ['tok2vec', 'ner']
[2025-05-04 12:37:10,643] [INFO] Created vocabulary
[2025-05-04 12:37:10,644] [INFO] Finished initializing nlp object
[2025-05-04 12:37:16,392] [DEBUG] [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed and load th

In [19]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example

# 1. Load your trained model
nlp = spacy.load("output/model-best")  # or "output/model-last"

# 2. Load test data
doc_bin = DocBin().from_disk("test.spacy")
gold_docs = list(doc_bin.get_docs(nlp.vocab))

# 3. Generate predictions and prepare examples
examples = []
for gold_doc in gold_docs:
    pred_doc = nlp(gold_doc.text)  # Generate predictions
    example = Example(predicted=pred_doc, reference=gold_doc)
    examples.append(example)

# 4. Score using the nlp.evaluate() method
results = nlp.evaluate(examples)

# 5. Print results
print(f"NER F1: {results['ents_f']:.2f}")
print(f"Precision: {results['ents_p']:.2f}")
print(f"Recall: {results['ents_r']:.2f}")
print("\nPer-type scores:", results["ents_per_type"])


NER F1: 0.82
Precision: 0.81
Recall: 0.83

Per-type scores: {'B-ORG': {'p': 0.7693156732891833, 'r': 0.85, 'f': 0.8076477404403244}, 'B-PER': {'p': 0.832282471626734, 'r': 0.8527131782945736, 'f': 0.8423739629865985}, 'B-LOC': {'p': 0.8296382730455076, 'r': 0.8876404494382022, 'f': 0.8576598311218335}, 'I-LOC': {'p': 0.897196261682243, 'r': 0.7272727272727273, 'f': 0.8033472803347281}, 'B-MISC': {'p': 0.7229916897506925, 'r': 0.7653958944281525, 'f': 0.7435897435897435}, 'I-MISC': {'p': 0.6222222222222222, 'r': 0.5490196078431373, 'f': 0.5833333333333334}, 'I-ORG': {'p': 0.7282051282051282, 'r': 0.7573333333333333, 'f': 0.7424836601307189}, 'I-PER': {'p': 0.9133574007220217, 'r': 0.8784722222222222, 'f': 0.895575221238938}}
